# 🤪 变分自编码器（VAE）——CelebA 人脸

本 notebook 将逐步介绍如何在 CelebA 人脸数据集 上训练你自己的 变分自编码器（VAE）

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils as vutils

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import os

## 0. Parameters <a name="parameters"></a>

In [3]:
IMAGE_SIZE = 32
CHANNELS = 3
BATCH_SIZE = 128
NUM_FEATURES = 128
Z_DIM = 200
LEARNING_RATE = 5e-4
EPOCHS = 10
BETA = 2000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## 1. Prepare the data <a name="prepare"></a>

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),           # [0,1]
])

train_dataset = datasets.ImageFolder(
    root="./data/celeba-dataset/img_align_celeba",
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    drop_last=True
)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './data/img_align_celeba'

In [7]:
# 显示一批人脸
images, _ = next(iter(train_loader))
grid = vutils.make_grid(images[:16], nrow=4)
plt.figure(figsize=(6,6))
plt.imshow(grid.permute(1,2,0))
plt.axis("off")
plt.show()

NameError: name 'train_loader' is not defined

## 2. Build the variational autoencoder <a name="build"></a>

In [ ]:
def reparameterize(mu, logvar):
    eps = torch.randn_like(mu)
    return mu + torch.exp(0.5 * logvar) * eps

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, NUM_FEATURES, 3, 2, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.Conv2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.Conv2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.Conv2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),
        )
        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(NUM_FEATURES * 2 * 2, Z_DIM)
        self.fc_logvar = nn.Linear(NUM_FEATURES * 2 * 2, Z_DIM)

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        z = reparameterize(mu, logvar)
        return mu, logvar, z

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(Z_DIM, NUM_FEATURES * 2 * 2),
            nn.BatchNorm1d(NUM_FEATURES * 2 * 2),
            nn.LeakyReLU()
        )

        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(NUM_FEATURES, NUM_FEATURES, 3, 2, 1, 1),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(NUM_FEATURES, CHANNELS, 3, 2, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, z):
        x = self.fc(z)
        x = x.view(-1, NUM_FEATURES, 2, 2)
        return self.deconv(x)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        mu, logvar, z = self.encoder(x)
        recon = self.decoder(z)
        return mu, logvar, recon

In [ ]:
vae = VAE().to(DEVICE)
optimizer = torch.optim.Adam(vae.parameters(), lr=LEARNING_RATE)

## 3. Train the variational autoencoder <a name="train"></a>

In [ ]:
def vae_loss(x, recon, mu, logvar):
    recon_loss = BETA * F.mse_loss(recon, x, reduction="mean")
    kl = -0.5 * torch.mean(
        torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)
    )
    return recon_loss + kl, recon_loss, kl

In [ ]:
vae.train()
for epoch in range(EPOCHS):
    total = 0
    for x, _ in train_loader:
        x = x.to(DEVICE)
        mu, logvar, recon = vae(x)
        loss, r, kl = vae_loss(x, recon, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total += loss.item()

    print(f"Epoch {epoch+1}: loss={total/len(train_loader):.4f}")

## 4. Latent space distribution

In [ ]:
vae.eval()
with torch.no_grad():
    x, _ = next(iter(train_loader))
    x = x.to(DEVICE)
    mu, logvar, z = vae.encoder(x)

z = z.cpu().numpy()

In [ ]:
x_axis = np.linspace(-3, 3, 100)
plt.figure(figsize=(20,5))
for i in range(50):
    plt.subplot(5,10,i+1)
    plt.hist(z[:,i], bins=20, density=True)
    plt.plot(x_axis, norm.pdf(x_axis))
    plt.axis("off")
plt.show()

## 5. Generate new faces <a name="decode"></a>

In [ ]:
with torch.no_grad():
    z_sample = torch.randn(30, Z_DIM).to(DEVICE)
    samples = vae.decoder(z_sample).cpu()

In [ ]:
grid = vutils.make_grid(samples, nrow=10)
plt.figure(figsize=(18,5))
plt.imshow(grid.permute(1,2,0))
plt.axis("off")
plt.show()

## 6. Manipulate the images <a name="manipulate"></a>

In [ ]:
# Load the label dataset
attributes = pd.read_csv("/app/data/celeba-dataset/list_attr_celeba.csv")
print(attributes.columns)
attributes.head()

In [ ]:
# Load the face data with label attached
LABEL = "Blond_Hair"  # <- Set this label
labelled_test = utils.image_dataset_from_directory(
    "/app/data/celeba-dataset/img_align_celeba",
    labels=attributes[LABEL].tolist(),
    color_mode="rgb",
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
)

labelled = labelled_test.map(lambda x, y: (preprocess(x), y))

In [ ]:
# Find the attribute vector
attribute_vec = get_vector_from_label(labelled, vae, Z_DIM, LABEL)

In [ ]:
# Add vector to images
add_vector_to_images(labelled, vae, attribute_vec)

In [ ]:
morph_faces(labelled, vae)